## Part 1

Steps involved :- 
1. Loading Training/Testing Data
2. Formatting into structured JSON format
3. Saving File named "" "updated" + Filename ""

In [6]:
#Loading JSON Library
import json

In [4]:
filename = "f.json"

In [5]:
with open(filename,"r", encoding="utf8") as f:
    data = f.read()

In [6]:
def savetoJSONFormat(data):
    index = data.find("\n")
    data[:index+2]
    
    # no "," after every sentence and overall [] are missing for proper json input
    datareplaced = data.replace("\n",",")
    datareplaced.find("\n")
    
    #above code add "," at last sentence so removing it
    index = len(datareplaced)
    datareplaced = datareplaced[:index-1]
    
    
    with open("updated "+filename,"w+") as f:
        f.write("[" + datareplaced + "]")
        print("File saved :- ", "updated "+filename)
    return 

In [7]:
savetoJSONFormat(data)

File saved :-  updated f.json


# Part 2

Steps invloved :-
1. Loading JSON File of  Traning and Testing Data
2. Formating JSON file to required format for training

{ Loading the New Testing Set (Using On Unseen Data Set)  to avoid overfitting of the training data }

In [30]:
trainfile = "f.json"

In [31]:
with open("updated "+trainfile) as train_data:
    train = json.load(train_data)

In [32]:
TRAIN_DATA = []

for data in train:
    ents = [tuple(entity) for entity in data['labels']]
    TRAIN_DATA.append((data['text'],{'entities':ents}))

print(TRAIN_DATA[0])


('Gujarmal Modi 9 August 1902  22 January 1976 was an Indian industrialist and philanthropist, who co-established the Modi Group of companies and the industrial city of Modinagar in 1933, along with his brother Kedar Nath Modi', {'entities': [(0, 13, 'Person'), (209, 224, 'Person'), (167, 176, 'Location')]})


In [7]:
testfile =  "train1.json"

In [8]:
with open("updated "+testfile) as test_data:
    test = json.load(test_data) 

In [9]:
TEST_DATA = []

for data in test:
    ents = [tuple(entity) for entity in data['labels']]
    TEST_DATA.append((data['text'],{'entities':ents}))
    
print(TEST_DATA[0]) 

('PM Modi and his officials will be holding talks and inaugurate a few defence projects', {'entities': [(0, 7, 'Narendra Modi')]})


# Part  3

Steps involved :- 
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [10]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer


In [15]:
def train_spacy(data,iterations,droprate = 0.5,modelName = "modelTrained"):
    
    # Loading Traning data
    TRAIN_DATA = data
    
    
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score 
            results = evaluate(nlp, TEST_DATA)
            print("Current Score :-",results["ents_f"])

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:
                    modelfile = modelName
                    pnlp = spacy.load(modelfile)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                # Save our trained Model if the score if grater than best score else no change in previous model
                nlp.to_disk(modelName)
                
    print("-----Best Model is Saved-----")
    return 


In [16]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores


In [12]:
def loadNERModel(modelName = "modelTrained"):
    pnlp = spacy.load(modelName)
    return pnlp

In [13]:
def f1score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    print("F1 score off Model is :-",f1score)

In [24]:
# traning the model with 50 iterations
train_spacy(TRAIN_DATA, 100,droprate = 0.55, modelName = "nm2.0")

Starting iteration 0
{'ner': 53.8163348590263}
Current Score :- 14.705882352941178
Best Sccore :-  71.280276816609
Starting iteration 1
{'ner': 22.770258148378126}
Current Score :- 26.08695652173913
Best Sccore :-  71.280276816609
Starting iteration 2
{'ner': 20.729032089713275}
Current Score :- 30.943396226415093
Best Sccore :-  71.280276816609
Starting iteration 3
{'ner': 17.293015892204004}
Current Score :- 50.174216027874564
Best Sccore :-  71.280276816609
Starting iteration 4
{'ner': 17.3278846234176}
Current Score :- 49.30555555555556
Best Sccore :-  71.280276816609
Starting iteration 5
{'ner': 16.088963218377096}
Current Score :- 55.714285714285715
Best Sccore :-  71.280276816609
Starting iteration 6
{'ner': 16.362135499011483}
Current Score :- 56.74740484429065
Best Sccore :-  71.280276816609
Starting iteration 7
{'ner': 13.918331864765939}
Current Score :- 54.21245421245422
Best Sccore :-  71.280276816609
Starting iteration 8
{'ner': 11.847385018433267}
Current Score :- 54.074

In [14]:
# loading the saved model
pnlp = loadNERModel("nm2.0")

In [17]:
# calculating the score of the model
f1score(pnlp,TEST_DATA)

F1 score off Model is :- 71.47766323024054


In [18]:
from spacy import displacy

In [23]:
testcase = pnlp(TEST_DATA[0][0])
displacy.render(testcase, style='ent', jupyter=True)

In [41]:
for i in range(0,30):
    try:        
        testcase = pnlp(TEST_DATA[i][0])
        displacy.render(testcase, style='ent', jupyter=True)
        print("--")
    except:
        a=None

--


--


--


--


E:\Anaconda3\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

--


--


--


--


--


'PM Modi and his officials will be holding talks and inaugurate a few defence projects.\nPrime Minister Modi landed in the country capital Male where he is expected to hold talks with President Solih.\nFriendship forever!PM @narendramodi arrives in Male capital of Maldives to a warm reception by Foreign Minister @abdulla_shahid.\nMaldives is also planning to honour PM Modi with the Order of Nishanizzuddeen a prestigious award in the country.\n@narendramodi the Maldives highest honour accorded to foreign dignitaries.\nPrime Minister Modi will also be addressing the Maldives Parliament on Saturday.\n#WATCH Prime Minister Narendra Modi inspects guard of honour at Republic Square in Male; President of Maldives Ibrahim Mohamed Solih also present.\ncom/RJ59DGeeoN ANI @ANI June 8 2019Modi is visiting Maldives in his first bilateral visit after retaining power in the Lok Sabha polls.\nModi has reached the Indian Ocean archipelago in his first bilateral visit after retaining power in the Lok Sa

In [ ]:
trainstring = TRAIN_DATA[0][0]

In [38]:
for i in range(0,10):
    testcase = pnlp(TRAIN_DATA[i][0])
    displacy.render(testcase, style='ent', jupyter=True)
    print("--")

--


--


--


--


--


--


--


--


--


--
